# Étiquetage morphosyntaxique

## Définition

Opération par laquelle un programme associe automatiquement à un mot des étiquettes grammaticales, comme :
- la partie du discours (*N*)
- le genre (*NF*)
- le nombre (*NFP*)
- …

Elle intervient après celle de segmentation en mots et se positionne comme pré-requis pour l’analyse syntaxique de surface.

Le résultat est un couple (mot, étiquette) :
- *Le petit chat boit du lait.*
- *Le/DET petit/ADJ chat/N boit/V du/DET lait/N ./PONCT*

Certaines étiquettes étant en concurrence pour un mot, leur attribution est rarement aisée. 

### Exemple d’énoncé ambigu

Considérons la phrase ci-dessous :

> La petite brise la glace.

Elle peut s’interpréter de deux façons : soit on imagine une petite fille en colère brisant un miroir, soit on se place dans un contexte hivernal où le vent fait trembler notre héroïne :

```mermaid
flowchart TB
    subgraph "Les déboires d’une aventurière"
        direction TB
        A2("La"):::formal-->At2["DET"]:::academic
        B2("petite"):::formal-->Bt2["ADJ"]:::academic
        C2("brise"):::formal-->Ct2["N"]:::academic
        D2("la"):::formal-->Dt2["PRO"]:::academic
        E2("glace"):::formal-->Et2["V"]:::academic
        F2("."):::formal-->Ft2["PONCT"]:::academic
    end
    subgraph "La colère d’une petite fille"
        direction TB
        A("La"):::formal-->At1["DET"]:::academic
        B("petite"):::formal-->Bt1["N"]:::academic
        C("brise"):::formal-->Ct1["V"]:::academic
        D("la"):::formal-->Dt1["DET"]:::academic
        E("glace"):::formal-->Et1["N"]:::academic
        F("."):::formal-->Ft1["PONCT"]:::academic
    end
    classDef formal fill:#fff,stroke:#D68738,color:#D68738
    classDef academic fill:#032B4F,stroke:#032B4F,color:#fff
```

Si cet énoncé est caractéristique des difficultés qu’un étiqueteur peut rencontrer, la majorité des mots et des phrases du français est ambiguë.

## Étiquetage et décision

Pour un étiqueteur, l’analyse de la phrase revient à prendre une décision entre deux voies possibles : l’une ouverte par l’interprétation de *petite* comme un nom et l’autre comme un adjectif.

```mermaid
flowchart LR
    A("La
    DET"):::academic-->B("petite
    N"):::formal
    B-->C("brise
    V"):::formal
    C-->D("la
    DET"):::formal
    D-->E("glace
    N"):::formal
    E-->F(".
    PONCT"):::academic
    A-->G("petite
    ADJ"):::academic
    G-->H("brise
    N"):::academic
    H-->I("la
    PRO"):::academic
    I-->J("glace
    V"):::academic
    J-->F
    classDef formal fill:#fff,stroke:#D68738,color:#D68738
    classDef academic fill:#032B4F,stroke:#032B4F,color:#fff
```

Ces deux voies ne sont pas équiprobables et l’interprétation du mot *petite* dépendra fortement du corpus d’apprentissage.

### Analyse d’un unigramme

Pour un mot *w* donné, on peut déterminer son étiquette *t* grâce à sa fréquence d’apparition dans le corpus :

$$
P(t|w) = \frac{F(w,t)}{F(w)}
$$

Si le mot *petite* apparaît 38 fois dont 12 en tant que nom, la probabilité de le voir avec l’étiquette *ADJ* est de :

$$
P(ADJ|petite) = \frac{26}{38} = 0,6842
$$

À noter que la probabilité de *t* sachant *w* peut s’écrire comme un rapport entre la probabilité du couple mot/étiquette et la probabilité du mot :

$$
P(t|w) = \frac{P(t,w)}{P(w)}
$$

Dans notre exemple fictif, imaginons que nous avons entraîné notre étiqueteur sur un corpus de 1000 mots. La formule donnerait :

$$
P(ADJ|petite) = \frac{\frac{26}{1000}}{\frac{38}{1000}} = 0,6842
$$

De manière réciproque, il serait possible de déduire le mot sachant l’étiquette. Toutefois, la précision dans ce sens sera bien moindre : quand on hésite seulement entre quelques étiquettes pour un mot, la quantité de mots possibles pour une étiquette est bien plus importante.

Ce modèle est jugé simpliste : non seulement il choisit parmi plusieurs possibilités la plus probable ($P(ADJ|petite) > P(N|petite)$) mais en plus il ne tient pas compte de l’antériorité. Au mot suivant, *brise*, il pourrait choisir d’attribuer l’étiquette *V*.

### Étiquetage *n*-gramme

#### Définition de la tâche

L’idée sous-jacente des étiqueteurs *n*-gramme est que l’étiquette d’un mot dépend des *n-1* mots qui le précèdent. Dans la pratique, comme les ressources nécessaires pour le calcul des probabilités croissent avec *n* et que la fiabilité des résultats diminue, notamment en raison d’un phénomène de dilution, on se contente d’une faible valeur de *n*.

Trois hypothèses sont retenues :

- l’approche bayésienne  
$P(t|w) = \frac{P(w|t) \cdot P(t)}{P(w)}$
- l’horizon limité de Markov  
$P(t_n|t_{1,n-1}) = P(t_n|t_{n-1})$
- la probablité d’apparition d’un mot pour une étiquette ne dépend que de son étiquette  
$P(w_i|t_{1,n}) = P(w_i|t_i)$

#### Paramètres de résolution

Soient deux ensembles *W* et *T* pour les mots et les étiquettes. À une séquence de *n* mots est associée une séquence de *n* étiquettes de telle manière que :

$$
\begin{align}
    w_{1,n} &= w_1 w_2 \ldots w_n\\
    t_{1,n} &= t_1 t_2 \ldots t_n
\end{align}
$$

L’objectif, sachant $w_{1,n}$, est de déterminer $t_{1,n}$ en maximisant $P(t_{1,n}|w_{1,n})$, c’est-à-dire, après avoir calculé toutes les probabilités possibles, de ne conserver que la plus forte :

$$
t_{1,n} = max_{t_{1,n}} P(t_{1,n}|w_{1,n})
$$

Ce qui, d’après le théorème de Bayes, s’exprime en renversant la condition :

$$
\begin{align}
    P(t_{1,n}|w_{1,n}) &= \frac{P(w_{1,n}|t_{1,n}) \cdot P(t_{1,n})}{P(w_{1,n})}\\
    t_{1,n} &= \text{max} P(w_{1,n}|t_{1,n}) \cdot P(t_{1,n})
\end{align}
$$

Il reste deux probabilités à calculer pour lesquelles on applique les hypothèses retenues plus haut :

- $P(w_{1,n}|t_{1,n})$ : hypothèse selon laquelle un mot ne dépend que de son étiquette
- $P(t_{1,n})$ : hypothèse selon laquelle une étiquette dépend de ses *n* étiquettes précédentes

$$
\begin{align}
    P(w_{1,n}|t_{1,n}) &= P(w_1|t_1) \cdot P(w_2|t_2) \ldots P(w_n|t_n)\\
    P(t_{1,n}) &= P(t_1) \cdot P(t_2|t_1) \cdot P(t_3|t_2) \ldots P(t_n|t_{n−1})
\end{align}
$$

### Modèle de Markov caché d’ordre *N*

Le modèle décrit plus haut est appelé modèle de Markov caché d’ordre 1 (*Hidden Markov Model*, ou HMM) dans la mesure où la probabilité d’apparition d’une étiquette ne dépend que de la précédente. Pour un ordre plus élevé, par exemple pour un HMM d’ordre 2, nous aurons :

$$
P(t_{1,n}) = P(t_1) \cdot P(t_2|t_1) \cdot P(t_3|t_1 t_2) \ldots P(t_n|t_{n−2}t_{n−1})
$$

En appliquant un HMM, la tâche pour un étiquetage *n*-gramme se formulerait ainsi :

$$
t_{1,n} = \text{max} P(w_1|t_1) \cdot \prod_{i=1,n} P(t_i|t_{i-1}) \cdot P(w_i|t_i)
$$

### Estimation des probabilités avec un corpus d’apprentissage

La première étape consiste à calculer les probabilités d’émission pour chaque mot sachant son étiquette :

$$
P(w_i|t_i) = \frac{F(w_i, t_i)}{F(t_i)}
$$

Ensuite, on passe aux probabilités de transition :

$$
P(t_i|t_{i-1}) = \frac{F(t_{i-1} t_i)}{F(t_{i-1})}
$$

## Étiquetage manuel avec un HMM d’ordre 2

Soit la phrase à étiqueter :

> Lemmy joue de la basse.

### Apprentissage

Un corpus d’apprentissage fictif nous permet de dresser le tableau des probabilités d’apparition suivant :

|$w, t$|ADJ|ART|NC|NP|PRE|PRO|V|
|:-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|**Lemmy**|0|0|0|0.0001|0|0|0|
|**joue**|0|0|0.06|0|0|0|0.02|
|**de**|0|0.102|0|0|0.013|0|0|
|**la**|0|0.12|0.0002|0|0|0.14|0|
|**basse**|0.0015|0|0.0002|0|0|0|0|

Ainsi que, pour les bigrammes potentiels :

|$t|t_{-1}$|ADJ|ART|NC|NP|PRE|PRO|V|
|-|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|**ADJ**|0|0.0001|0.34|0|0|0.001|0|
|**ART**|0|0.32|0.09|0|0.63|0|0.51|
|**NC**|0|0.18|0.02|0.0001|0.001|0.03|0|
|**NP**|0|0|0|0|0|0|0|
|**PRE**|0|0|0.23|0|0|0|0.0001|
|**PRO**|0|0.001|0|0|0.22|0|0|
|**V**|0|0|0|0.48|0|0|0|

### Arbre des possibilités

Les calculs combinatoires étant trop nombreux, nous implémentons un arbre de décision pour les besoins de l’exercice.

Ci-dessous l’arbre des étiquetages possibles avec la séquence privilégiée (*Lemmy/NP joue/V de/ART la/ART basse/NC*) :

```mermaid
flowchart TB

    A("NP"):::academic-->B1("NC"):::formal
    A-->B2("V"):::academic

    B1-->C1("ART"):::formal
    B1-->C2("PRE"):::formal

    B2-->C3("ART"):::academic
    B2-->C4("PRE"):::formal

    C1-->D1("ART"):::formal
    C1-->D2("NC"):::formal
    C1-->D3("PRO"):::formal
    
    C2-->D4("ART"):::formal
    C2-->D5("NC"):::formal
    C2-->D6("PRO"):::formal
    
    C3-->D7("ART"):::academic
    C3-->D8("NC"):::formal
    C3-->D9("PRO"):::formal
    
    C4-->D10("ART"):::formal
    C4-->D11("NC"):::formal
    C4-->D12("PRO"):::formal

    D1-->E1("ADJ"):::formal
    D1-->E2("NC"):::formal

    D2-->E3("ADJ"):::formal
    D2-->E4("NC"):::formal

    D3-->E5("ADJ"):::formal
    D3-->E6("NC"):::formal

    D4-->E7("ADJ"):::formal
    D4-->E8("NC"):::formal

    D5-->E9("ADJ"):::formal
    D5-->E10("NC"):::formal

    D6-->E11("ADJ"):::formal
    D6-->E12("NC"):::formal

    D7-->E13("ADJ"):::formal
    D7-->E14("NC"):::academic

    D8-->E15("ADJ"):::formal
    D8-->E16("NC"):::formal

    D9-->E17("ADJ"):::formal
    D9-->E18("NC"):::formal

    D10-->E19("ADJ"):::formal
    D10-->E20("NC"):::formal

    D11-->E21("ADJ"):::formal
    D11-->E22("NC"):::formal

    D12-->E23("ADJ"):::formal
    D12-->E24("NC"):::formal

    classDef formal fill:#fff,stroke:#D68738,color:#D68738
    classDef academic fill:#032B4F,stroke:#032B4F,color:#fff
```

### Calcul des probabilités

Sachant que Lemmy ne peut être étiqueté qu’avec *NP*, la question revient à connaître la probabilité de la séquence *joue de la basse*. Nous identifions les bigrammes suivants : *Lemmy joue*, *joue de*, *de la* et *la basse*. Pour chacun, il convient de calculer toutes les possibilités d’étiquetage et conserver la valeur maximale.

#### Bigramme *Lemmy joue*

1. Séquence *NP, NC* :
   $$
   \begin{align}
   &= P(Lemmy|NP) \times P(joue|NC) \times P(NC|NP)\\
   &= 0.0001 \times 0.06 \times 0.0001\\
   &= 6 \times 10^{-10}
   \end{align}
   $$
2. Séquence *NP, V* :
   $$
   \begin{align}
   &= P(Lemmy|NP) \times P(joue|V) \times P(V|NP)\\
   &= 0.0001 \times 0.02 \times 0.48\\
   &= 9.6 \times 10^{-7}
   \end{align}
   $$

À ce stade, l’étiquetage privilégié est :

> Lemmy/NP joue/V

#### Bigramme *joue de*

1. Séquence *V, ART* :
   $$
   \begin{align}
   &= P(joue|V) \times P(de|ART) \times P(V|NP) \times P(ART|V)\\
   &= 0.02 \times 0.112 \times 0.48 \times 0.51\\
   &= 5.48352 \times 10^{-4}
   \end{align}
   $$
2. Séquence *V, PRE* :
   $$
   \begin{align}
   &= P(joue|V) \times P(de|PRE) \times P(V|NP) \times P(PRE|V)\\
   &= 0.02 \times 0.013 \times 0.48 \times 0.0001\\
   &= 1.248 \times 10^{-8}
   \end{align}
   $$

À ce stade, l’étiquetage privilégié est :

> Lemmy/NP joue/V de/ART

#### Bigramme *de la*

1. Séquence *ART, ART* :
   $$
   \begin{align}
   &= P(de|ART) \times P(la|ART) \times P(ART|V) \times P(ART|ART)\\
   &= 0.102 \times 0.12 \times 0.51 \times 0.32\\
   &= 1.997 \times 10^{-3}
   \end{align}
   $$
2. Séquence *ART, NC* :
   $$
   \begin{align}
   &= P(de|ART) \times P(la|NC) \times P(ART|V) \times P(NC|ART)\\
   &= 0.102 \times 0.0002 \times 0.32 \times 0.18\\
   &= 1.175 \times 10^{-6}
   \end{align}
   $$
3. Séquence *ART, PRO* :
   $$
   \begin{align}
   &= P(de|ART) \times P(la|PRO) \times P(ART|V) \times P(PRO|ART)\\
   &= 0.102 \times 0.14 \times 0.51 \times 0.001\\
   &= 7.2828 \times 10^{-6}
   \end{align}
   $$

À ce stade, l’étiquetage privilégié est :

> Lemmy/NP joue/V de/ART la/ART

#### Bigramme *la basse*

1. Séquence *ART, ADJ* :
   $$
   \begin{align}
   &= P(la|ART) \times P(basse|ADJ) \times P(ART|ART) \times P(ADJ|ART)\\
   &= 0.12 \times 0.0015 \times 0.32 \times 0.0001\\
   &= 5.76 \times 10^{-9}
   \end{align}
   $$
2. Séquence *ART, NC* :
   $$
   \begin{align}
   &= P(la|ART) \times P(basse|NC) \times P(ART|ART) \times P(NC|ART)\\
   &= 0.12 \times 0.0002 \times 0.32 \times 0.18\\
   &= 1.3824 \times 10^{-5}
   \end{align}
   $$

À présent, nous pouvons conclure que l’étiquetage le plus probable est :

> Lemmy/NP joue/V de/ART la/ART basse/NC